# Q&A against a SQL Database

Now that we know (from the prior Notebook) how to query tabular data on a CSV file, let's try now to keep the data at is source and ask questions directly to a SQL Database.
The goal of this notebook is to demonstrate how a LLM so advanced as GPT-4 can understand a human question and translate that into a SQL query to get the answer. 

We will be using the Azure SQL Server that you created on the initial deployment. The server should be created on the Resource Group where the Azure Cognitive Search service is located.

Let's begin..

In [1]:
import os
import pandas as pd
import pyodbc
from langchain.chat_models import AzureChatOpenAI
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain import SQLDatabaseChain
from langchain.chains import SQLDatabaseSequentialChain
from langchain.agents import AgentExecutor

from app.prompts import MSSQL_PROMPT, MSSQL_AGENT_PREFIX, MSSQL_AGENT_FORMAT_INSTRUCTIONS

from IPython.display import Markdown, HTML, display  

def printmd(string):
    display(Markdown(string))

# Don't mess with this unless you really know what you are doing
AZURE_OPENAI_API_VERSION = "2023-03-15-preview"

# Change these below with your own services credentials
AZURE_OPENAI_ENDPOINT = "ENTER YOUR VALUE"
AZURE_OPENAI_API_KEY = "ENTER YOUR VALUE"
SQL_SERVER_ENDPOINT = "ENTER YOUR VALUE"
SQL_SERVER_DATABASE = "ENTER YOUR VALUE"
SQL_SERVER_USERNAME = "ENTER YOUR VALUE"
SQL_SERVER_PASSWORD = "ENTER YOUR VALUE"

In [2]:
# Set the ENV variables 
os.environ["OPENAI_API_BASE"] = os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
os.environ["OPENAI_API_KEY"] = os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"] = AZURE_OPENAI_API_VERSION
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["SQL_SERVER_ENDPOINT"] = SQL_SERVER_ENDPOINT
os.environ["SQL_SERVER_DATABASE"] = SQL_SERVER_DATABASE
os.environ["SQL_SERVER_USERNAME"] = SQL_SERVER_USERNAME
os.environ["SQL_SERVER_PASSWORD"] = SQL_SERVER_PASSWORD

# Install MS SQL DB driver in your machine

We need the driver installed on this compute in order to talk to the SQL DB, so run the below cell once<br>
Reference: https://learn.microsoft.com/en-us/sql/connect/odbc/linux-mac/installing-the-microsoft-odbc-driver-for-sql-server?view=sql-server-ver16&tabs=ubuntu18-install%2Calpine17-install%2Cdebian8-install%2Credhat7-13-install%2Crhel7-offline

In [3]:
# !sudo ./download_odbc_driver.sh

# Load Azure SQL DB with the Covid Tracking CSV Data

The Azure SQL Database is currently empty, so we need to fill it up with data. Let's use the same data on the Covid CSV filed we used on the prior Notebook, that way we can compare results and methods. 
For this, you will need to type below the credentials you used at creation time.

In [23]:
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL

db_config = {
                'drivername': 'mssql+pyodbc',
                'username': os.environ["SQL_SERVER_USERNAME"] +'@'+ os.environ["SQL_SERVER_ENDPOINT"],
                'password': os.environ["SQL_SERVER_PASSWORD"],
                'host': os.environ["SQL_SERVER_ENDPOINT"],
                'port': 1433,
                'database': os.environ["SQL_SERVER_DATABASE"],
                'query': {'driver': 'ODBC Driver 17 for SQL Server'}
            }

# Create a URL object for connecting to the database
db_url = URL.create(**db_config)

# Print the resulting URL string
# print(db_url)

# Connect to the Azure SQL Database using the URL string
engine = create_engine(db_url)

# Test the connection
try:
    conn = engine.connect()
    print("Connection successful!")
    result = engine.execute("SELECT @@Version")
    for row in result:
        print(row)
    conn.close()
    
except OperationalError:
    print("Connection failed.")

Connection successful!
('Microsoft SQL Azure (RTM) - 12.0.2000.8 \n\tMar 30 2023 16:38:37 \n\tCopyright (C) 2022 Microsoft Corporation\n',)


In [5]:
# Read CSV file into a pandas dataframe
csv_path = "./data/all-states-history.csv"
df = pd.read_csv(csv_path).fillna(value = 0)

# Infer column names and data types
column_names = df.columns.tolist()
column_types = df.dtypes.to_dict()

# Generate SQL statement to create table
table_name = 'covidtracking'

create_table_sql = f"CREATE TABLE {table_name} ("
for name, dtype in column_types.items():
    if dtype == 'object':
        create_table_sql += f"{name} VARCHAR(255), "
    elif dtype == 'int64':
        create_table_sql += f"{name} INT, "
    elif dtype == 'float64':
        create_table_sql += f"{name} FLOAT, "
    elif dtype == 'bool':
        create_table_sql += f"{name} BIT, "
    elif dtype == 'datetime64[ns]':
        create_table_sql += f"{name} DATETIME, "
create_table_sql = create_table_sql[:-2] + ")"

try:
    engine.execute(create_table_sql)
except Exception as e:
    print(e)
    
# Insert data into SQL Database
try:
    df.to_sql(table_name, con=engine, if_exists='fail', index=False)
except Exception as e:
    print(e)

(pyodbc.ProgrammingError) ('42S01', "[42S01] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]There is already an object named 'covidtracking' in the database. (2714) (SQLExecDirectW)")
[SQL: CREATE TABLE covidtracking (date VARCHAR(255), state VARCHAR(255), death FLOAT, deathConfirmed FLOAT, deathIncrease INT, deathProbable FLOAT, hospitalized FLOAT, hospitalizedCumulative FLOAT, hospitalizedCurrently FLOAT, hospitalizedIncrease INT, inIcuCumulative FLOAT, inIcuCurrently FLOAT, negative FLOAT, negativeIncrease INT, negativeTestsAntibody FLOAT, negativeTestsPeopleAntibody FLOAT, negativeTestsViral FLOAT, onVentilatorCumulative FLOAT, onVentilatorCurrently FLOAT, positive FLOAT, positiveCasesViral FLOAT, positiveIncrease INT, positiveScore INT, positiveTestsAntibody FLOAT, positiveTestsAntigen FLOAT, positiveTestsPeopleAntibody FLOAT, positiveTestsPeopleAntigen FLOAT, positiveTestsViral FLOAT, recovered FLOAT, totalTestEncountersViral FLOAT, totalTestEncountersViralIncrease INT, to

# Query with LLM

In [6]:
# Create or LLM Langchain object using GPT-4 deployment
llm = AzureChatOpenAI(deployment_name="gpt-4", temperature=0, max_tokens=500)

In [7]:
# Let's use a type of Chain made for this type of SQL work.  
db = SQLDatabase.from_uri(db_url)
db_chain = SQLDatabaseChain(llm=llm, database=db, prompt=MSSQL_PROMPT, verbose=True)

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/langchain/chains/sql_database/base.py:63: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [8]:
# Let's check our prompt we created in app/prompts.py
print(db_chain.prompt.template)


You are an MS SQL expert. Given an input question, first create a syntactically correct MS SQL query to run, then look at the results of the query and return the answer to the input question.

Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the TOP clause as per MS SQL. You can order the results to return the most informative data in the database.

Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in square brackets ([]) to denote them as delimited identifiers.

Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.

**Do not use double quotes on the SQL query**. 

Your response should be in Markdown.

** ALWAYS before giving the Final Answer, try another method**. Then reflect on the answers of th

In [9]:
# Natural Language question (query)
query_str = 'How may patients in total were hospitalized during July 2020 in Texas, and nationwide?'

In [10]:
printmd(db_chain(query_str)['result'])



> Entering new SQLDatabaseChain chain...
How may patients in total were hospitalized during July 2020 in Texas, and nationwide?
SQLQuery:SELECT [state], SUM([hospitalizedIncrease]) as TotalHospitalized
FROM covidtracking
WHERE [date] LIKE '2020-07%' AND ([state] = 'TX' OR [state] = 'US')
GROUP BY [state]
SQLResult: [('TX', 0)]
Answer:In July 2020, there were 0 patients hospitalized in Texas. Nationwide data is not available.

Explanation:
I queried the covidtracking table for the state and the sum of hospitalizedIncrease column where the date starts with '2020-07' and the state is either 'TX' or 'US'. The query returned a list of tuples with the total number of patients hospitalized in Texas during July 2020. The result shows that there were 0 patients hospitalized in Texas during that period. Nationwide data is not available in the table. I used the following query:

```sql
SELECT [state], SUM([hospitalizedIncrease]) as TotalHospitalized
FROM covidtracking
WHERE [date] LIKE '2020-07

In July 2020, there were 0 patients hospitalized in Texas. Nationwide data is not available.

Explanation:
I queried the covidtracking table for the state and the sum of hospitalizedIncrease column where the date starts with '2020-07' and the state is either 'TX' or 'US'. The query returned a list of tuples with the total number of patients hospitalized in Texas during July 2020. The result shows that there were 0 patients hospitalized in Texas during that period. Nationwide data is not available in the table. I used the following query:

```sql
SELECT [state], SUM([hospitalizedIncrease]) as TotalHospitalized
FROM covidtracking
WHERE [date] LIKE '2020-07%' AND ([state] = 'TX' OR [state] = 'US')
GROUP BY [state]
```

### To use or not use Agents

As you can see above we achieved our goal of Question->SQL without using an Agent, we did it just by using a clever prompt. (If you want to see the different kind of prompts templates that come with langchain for sql chain, you can check it out [HERE](https://github.com/hwchase17/langchain/blob/master/langchain/chains/sql_database/prompt.py)). **So the question is, why do we need an Agent then?**

**This is why**: As we explained on the prior Notebook, an agent is a process in which the LLM self-asks about what approach and steps to take, questions the validity of the results and if sure, provides the answer. The SQLDatabaseChain doesn't do all this analysis, but instead tries a one-shot query in order to answer the question, which is good! but not enough for complex questions. That's why it couldn't solve the part about nationwide, it needs multiple steps in order to solve the problem, one query is not enough.
Notice that it did't pay atention to the prompt where we explicitly say to try two methods and reflect on the answer.

Let's use an agent now and see how ReAct framework solves the problem

In [18]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent_executor = create_sql_agent(
    prefix=MSSQL_AGENT_PREFIX,
    format_instructions = MSSQL_AGENT_FORMAT_INSTRUCTIONS,
    llm=llm,
    toolkit=toolkit,
    verbose=True
)

In [19]:
# As we know by now, Agents use expert/tools. Let's see which are the tools for this SQL Agent
agent_executor.agent.allowed_tools

['query_sql_db', 'schema_sql_db', 'list_tables_sql_db', 'query_checker_sql_db']

In [20]:
# And let's see now our clever crafted prompt
printmd(agent_executor.agent.llm_chain.prompt.template)



You are an agent designed to interact with a SQL database.
## Instructions:
- Given an input question, create a syntactically correct mssql query to run, then look at the results of the query and return the answer.
- Unless the user specifies a specific number of examples they wish to obtain, **ALWAYS** limit your query to at most 10 results.
- You can order the results by a relevant column to return the most interesting examples in the database.
- Never query for all the columns from a specific table, only ask for the relevant columns given the question.
- You have access to tools for interacting with the database.
- You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.
- DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.
- **ALWAYS before giving the Final Answer, try another method**. Then reflect on the answers of the two methods you did and ask yourself if it answers correctly the original question. If you are not sure, try another method.
If the runs does not give the same result, reflect and try again until you have two runs that have the same result. If you still cannot arrive to a consistent result, say that you are not sure of the answer. But, if you are sure of the correct answer, create a beautiful and thorough response using Markdown. DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE, ONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE. 
- ALWAYS, as part of your final answer, explain how you got to the answer on a section that starts with: "Explanation:". Include the SQL query as part of the explanation section.
- If the question does not seem related to the database, just return "I don't know" as the answer.
- Only use the below tools. Only use the information returned by the below tools to construct your final answer.

## Tools:



query_sql_db: 
    Input to this tool is a detailed and correct SQL query, output is a result from the database.
    If the query is not correct, an error message will be returned.
    If an error is returned, rewrite the query, check the query, and try again.
    
schema_sql_db: 
    Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables.
    Be sure that the tables actually exist by calling list_tables_sql_db first!

    Example Input: "table1, table2, table3"
    
list_tables_sql_db: Input is an empty string, output is a comma separated list of tables in the database.
query_checker_sql_db: 
    Use this tool to double check if your query is correct before executing it.
    Always use this tool before executing a query with query_sql_db!
    



## Use the following format:

Question: the input question you must answer. 
Thought: you should always think about what to do. 
Action: the action to take, should be one of [query_sql_db, schema_sql_db, list_tables_sql_db, query_checker_sql_db]. 
Action Input: the input to the action. 
Observation: the result of the action. 
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer. 
Final Answer: the final answer to the original input question. 

Example of Final Answer:
<=== Beginning of example

Action: query_sql_db
Action Input: SELECT TOP (10) [death] FROM covidtracking WHERE state = 'TX' AND date LIKE '2020%'
Observation: [(27437.0,), (27088.0,), (26762.0,), (26521.0,), (26472.0,), (26421.0,), (26408.0,)]
Thought:I now know the final answer
Final Answer: There were 27437 people who died of covid in Texas in 2020.

Explanation:
I queried the `covidtracking` table for the `death` column where the state is 'TX' and the date starts with '2020'. The query returned a list of tuples with the number of deaths for each day in 2020. To answer the question, I took the sum of all the deaths in the list, which is 27437. 
I used the following query

```sql
SELECT [death] FROM covidtracking WHERE state = 'TX' AND date LIKE '2020%'"
```
===> End of Example



Begin!

Question: {input}
Thought: I should look at the tables in the database to see what I can query.
{agent_scratchpad}

In [14]:
for i in range(5):
    try:
        response = agent_executor.run(query_str)
        break
    except:
        response = "Error too many failed retries"
        continue
        
printmd(response)



> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: ""
Observation: covidtracking
Thought:I should check the schema of the covidtracking table to see what columns are available.
Action: schema_sql_db
Action Input: "covidtracking"
Observation: 
CREATE TABLE covidtracking (
	date VARCHAR(max) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	state VARCHAR(max) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	death FLOAT(53) NULL, 
	[deathConfirmed] FLOAT(53) NULL, 
	[deathIncrease] BIGINT NULL, 
	[deathProbable] FLOAT(53) NULL, 
	hospitalized FLOAT(53) NULL, 
	[hospitalizedCumulative] FLOAT(53) NULL, 
	[hospitalizedCurrently] FLOAT(53) NULL, 
	[hospitalizedIncrease] BIGINT NULL, 
	[inIcuCumulative] FLOAT(53) NULL, 
	[inIcuCurrently] FLOAT(53) NULL, 
	negative FLOAT(53) NULL, 
	[negativeIncrease] BIGINT NULL, 
	[negativeTestsAntibody] FLOAT(53) NULL, 
	[negativeTestsPeopleAntibody] FLOAT(53) NULL, 
	[negativeTestsViral] FLOAT(53) NULL, 
	[onVentilatorCumulative] FLOAT(

There were 0 patients hospitalized in Texas and 63,105 patients hospitalized nationwide during July 2020.

Explanation:
I queried the `covidtracking` table for the `hospitalizedIncrease` column where the date is between '2020-07-01' and '2020-07-31' for Texas and nationwide. I summed the `hospitalizedIncrease` column to get the total number of patients hospitalized during July 2020. For Texas, the query returned 0 patients, and for nationwide, the query returned 63,105 patients. I used the following queries:

```sql
SELECT SUM(hospitalizedIncrease) as TotalHospitalized FROM covidtracking WHERE date >= '2020-07-01' AND date <= '2020-07-31' AND state = 'TX'
```

```sql
SELECT SUM(hospitalizedIncrease) as TotalHospitalized FROM covidtracking WHERE date >= '2020-07-01' AND date <= '2020-07-31'
```

In [36]:
## This answer is way better than only using the SQL Chain
## IMPORTANT NOTE: Runing the above cell multiple times still will yield diferent results some times. The reason is:
## The column names are ambiguous, hence it is hard even for Humans to discern what are the right columns to use

# Summary

In this notebook, we achieved our goal of Asking a Question in natural language to a dataset located on a SQL Database.  We did this by using purely prompt engineering (Langchain does it for us) and the cognitive power of GPT-4.

This process shows why it is NOT necessary to move the data from its original source as long as the source has an API and a common language we can use to interface with. GPT-4 has been trained on the whole public Github corpus, so it can pretty much understand most of the coding and database query languages that exists out there. 

# NEXT

The Next Notebook will guide you on how we stick everything together. How do we use the features of the past notebooks and create a brain agent that can respond to any request accordingly.